# Module 4: Het verband tussen 2 kwalitatieve variabelen

Als casus lezen we de dataset `rlanders.csv` in (die synthetische data bevat, zie [de beschrijving](../data/rlanders.md)).

In [ ]:
# Importeren van de nodige packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests
import statsmodels.api as sm

import pandas as pd                                 # Dataframe
from pandas.api.types import CategoricalDtype       # Voor ordinale variabelen
import matplotlib.pyplot as plt                     # Basis visualisatie
from statsmodels.graphics.mosaicplot import mosaic  # Mozaïekdiagram
import seaborn as sns                               # Geavanceerde datavisualisatie
import altair as alt                                # Een alternatief visualisatiesysteem

In [ ]:
# Inlezen dataset + data preparation
rlanders = pd.read_csv('../data/rlanders.csv')
rlanders.set_index(['ID'])
rlanders.Gender = rlanders.Gender.astype('category')
likert_scale = CategoricalDtype(categories=[1,2,3,4,5], ordered=True)
rlanders.Survey = rlanders.Survey.astype(likert_scale)
# rlanders.info()
# rlanders.Survey.dtype

## Kruistabellen en visualisatietechnieken

Stel dat we willen weten of in de steekproef vrouwen en mannen (`Gender`) verschillend antwoordden op een enquêtevraag met een Likert-schaal (`Survey`). Als de verhoudingen tussen de antwoorden (1 t/m 5) voor vrouwen en mannen ongeveer gelijk zijn, dan zeggen we dat er geen verband is tussen beide variabelen. Als er wél een aanzienlijk verschil is, dan is er wel een verband. Nu is uiteraard de vraag vanaf wanneer we spreken van een *aanzienlijk* verschil...

Een eerste stap is het berekenen van een kruistabel. We voegen hier ook meteen de marginale totalen toe:

In [ ]:
pd.crosstab(rlanders.Survey, rlanders.Gender, margins=True)

Zo'n tabel zegt natuurlijk niet veel. Misschien moeten we de data eens visualiseren.

In [ ]:
sns.catplot(data=rlanders, x='Survey', hue='Gender', kind='count')

Wat opvalt is dat de vrouwen alvast niet de extreme antwoorden gaven. Er zijn wel beduidend minder vrouwen in de steekproef, dus misschien moeten we eerder naar de percentages kijken.

Een mozaïekdiagram is een grafische voorstelling van de kruistabel waar elke cel voorgesteld wordt door een tegel waarvan de oppervlakte proportioneel is met de frequentie van die cel t.o.v. het totaal aantal observaties.

Je kan dit een beetje vergelijken met hoe [WinDirStat](https://windirstat.net/) de grootte van bestanden op je harde schijf visualiseert.

Voor deze casus:

In [ ]:
from matplotlib import cm

# By default, mosaic() will color all columns the same. Here,
# we're going to give a separate color to each response (1-5).
# The response is kept in the label of each cell, a tuple of
# the two values that are represented in that cell (e.g. 
# ('Female', '2')). We will be using the "plasma" color map
# in this example.
plasma_colors = cm.get_cmap('plasma')
# Create a function that maps the cell key to the color we
# want to give it. We use the colormap function created above
# and give it a number between 0 and 1. We convert the second
# part of the key (values '1' to '5') to a number and divide
# by 5.
props = lambda key: {'color': plasma_colors(int(key[1])/5)}

mosaic(data=rlanders, index=['Gender', 'Survey'],
       gap=0.05, properties=props)

We zien hier duidelijk dat de mannen meer vertegenwoordigd zijn in de steekproef (bredere tegels).

Je kan ook de verdelingen van de gegeven antwoorden op de enquêtevraag vergelijken. Op het feit na dat de vrouwen geen extreme antwoorden gegeven hebben (1 en 5), lijken de verdelingen op het eerste zicht nog vrij goed overeen te komen.

## Chi-kwadraat en Cramér's V

Chi-kwadraat en Cramér's V zijn statistieken waarmee we kunnen bepalen of er een verband bestaat tussen twee kwalitatieve (categorische) variabelen.

De redenering gaat als volgt: als er geen verband is tussen de `Survey` en `Gender`, dan verwachten we dat de verhoudingen tussen de waarden van `Survey` gelijk zijn voor alle waarden van `Gender`. M.a.w. bij zowel de vrouwen als de mannen geeft hetzelfde percentage respondenten hetzelfde antwoord op de vraag.

In [ ]:
observed = pd.crosstab(rlanders.Survey, rlanders.Gender)
row_sums = observed.sum(axis=1)
col_sums = observed.sum()
n = row_sums.sum()

print(row_sums)
print(col_sums)
print(f'Aantal observaties: {n}')

Nu kunnen we berekenen hoe vaak we kunnen verwachten dat elke combinatie van `Gender` en `Survey` voorkomt in de steekproef. Bijvoorbeeld 114 van de 250 respondenten heeft "3" geantwoord, m.a.w. 114/250 = 0,456 (of 46,6%). Dan weten we dat zowel 46,6% van de vrouwen als 46,6% van de mannen "3" moet geantwoord hebben. Aangezien er 52 vrouwelijke respondenten waren, verwachten we dus dat er 52 x 0,456 = 23,712 "3" geantwoord hebben. In werkelijkheid waren het er ook 23. Bij de mannen verwachten we 198 x 0,456 = 90,288 (tegenover 91 in werkelijkheid).

Dit principe kunnen we veralgemenen naar elke cel in de tabel: $\frac{rijtotaal \times kolomtotaal}{n}$ (met $n$ het totaal aantal observaties).

De verwachte waarden kunnen we dan berekenen met de `outer()` functie:

In [ ]:
expected = np.outer(row_sums, col_sums) / n
expected

Kloppen de rij- en kolomsommen nog?

In [ ]:
exp_row_sums = np.sum(expected, axis=1)
exp_col_sums = np.sum(expected, axis=0)

print(f'Rijsommen  : {exp_row_sums}')
print(f'Kolomsommen: {exp_col_sums}')
print(f'Observaties: {exp_col_sums.sum()}')

Inderdaad, alles klopt. Wat is nu het verschil tussen de verwachte en geobserveerde waarden?

In [ ]:
expected - observed

Sommige geobserveerde waarden lijken dicht bij de verwachte te liggen (bv. voor "3"), andere liggen verderaf (bv. "2"). We moeten de verschillen echter ook in verhouding zien.

Een maat om de totale afwijking in een frequentietabel te bepalen, bestaat er uit om de verschillen tussen verwachte en geobserveerde waarden te kwadrateren (net zoals men bij variantie/standaardafwijking doet) en te delen door de verwachte waarde:

In [ ]:
diffs = (expected - observed)**2 / expected
print(diffs)

De som van al deze waarden wordt $\chi^2$ ("chi-kwadraat") genoemd:

$\chi^2 = \sum_i \frac{(o_i - e_i)^2}{e_i}$

met $o_i$ het aantal observaties in de $i$-de cel van de kruistabel en $e_i$ de verwachte frequentie. Vergelijk de formule met de code hierboven!

In [ ]:
chi_squared = diffs.values.sum()
print('χ² ≈ %.3f' %chi_squared)

Nu zegt deze waarde op zich nog steeds niet zo veel. Onder welke voorwaarden zeggen we dat er al dan niet een verband is tussen beide variabelen? Dat zal ook afhangen van de grootte van de tabel en het totaal aantal observaties. In een kruistabel met meer rijen/kolommen, zal je een grotere $\chi^2$ moeten hebben om te besluiten dat er een verband is.

[Cramér's V](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V) is een formule waarmee de $\chi^2$ kan genormaliseerd worden tot een waarde tussen 0 en 1 die onafhankelijk is van de tabelgrootte.

$V = \sqrt{\frac{\chi^2}{n(k-1)}}$

In [ ]:
dof = min(observed.shape) - 1
cramers_v = np.sqrt(chi_squared / (dof * n))
print(cramers_v)

Om een besluit te trekken uit dit getal, vergelijk je het met de waarden in onderstaande tabel:

| Cramér's V | Besluit            |
| :---:      | :---               |
| 0          | Geen verband       |
| 0.1        | Zwak verband       |
| 0.25       | Matig verband      |
| 0.50       | Sterk verband      |
| 0.75       | Zeer sterk verband |
| 1          | Volledig verband   |

Onze uitkomst voor Cramér's V wijst dus op een vrij zwak verband. Merk op dat Cramér's V erom gekend staat om in bepaalde gevallen te optimistisch te zijn over het verband tussen twee variabelen. Er bestaat een aangepaste formule om deze 'bias' weg te werken, maar die valt buiten het bereik van deze cursus.

Er bestaat echter nog een andere manier om te bepalen of de chi-kwadraat voldoende groot is om te besluiten dat er een verband is, nl. aan de hand van een statistische toets.

## De chi-kwadraat onafhankelijkheidstoets

Om een antwoord te vinden op de vraag vanaf wanneer de waarde van chi-kwadraat voldoende groot is om te veronderstellen dat er een verband is tussen twee variabelen, kunnen we gebruik maken van de *chi-kwadraat onafhankelijkheidstoets*.

### De chi-kwadraatverdeling

De waarde van $\chi^2$ volgt een specifieke stochastische verdeling die dan ook de $\chi^2$-verdeling genoemd wordt. Net zoals bij Student-t hangt de vorm van de dichtheidsfunctie af van het aantal vrijheidsgraden, wat in deze context $df = (r-1)\times(k-1)$ is (met $r$ het aantal rijen in de kruistabel en $k$ het aantal kolommen).

Hieronder vind je de code om de dichtheidsfunctie van de $\chi^2$-verdeling te plotten voor een aantal vrijheidsgraden:

In [ ]:
# Plot van de chi-kwadraatverdeling voor verschillende vrijheidsgraden
x = np.linspace(0, 10, num=100)
fig, tplot = plt.subplots(1, 1)
tplot.set_ylim([0, 0.8])
tplot.plot(x, stats.chi2.pdf(x, 1), label="df=1") 
tplot.plot(x, stats.chi2.pdf(x, 2), label="df=2") 
tplot.plot(x, stats.chi2.pdf(x, 3), label="df=3")
tplot.plot(x, stats.chi2.pdf(x, 8), label="df=5")
tplot.plot(x, stats.chi2.pdf(x, 30), label="df=10")
tplot.legend(loc='best')

Aan de hand van deze figuur, kan je een gelijkaardige redenering volgen als bij een rechtszijdige z-toets. Je kan een kritieke grenswaarde $g$ berekenen waarvoor geldt dat de oppervlakte onder de curve rechts van $g$ gelijk is aan ons gekozen significantieniveau $\alpha$. Als de teststatistiek $\chi^2$ groter is dan $g$, zeggen we dat de afwijkingen t.o.v. de verwachte waarden in de kruistabel té groot zijn om nog op toeval te berusten en dat we dus reden hebben om aan te nemen dat er een verband is tussen beide variabelen.

Je kan ook de overschrijdingskans $p$ berekenen, d.w.z. de kans dat je de afwijkingen die je in de steekproef ziet te wijten kunnen zijn aan toevallige steekproeffouten. Daarvoor bereken je de oppervlakte onder de dichtheidscurve rechts van de $\chi^2$. Als deze oppervlakte kleiner is dan $\alpha$, zeggen we eveneens dat de afwijkingen te groot zijn om veroorzaakt te worden door toevallige steekproeffouten en dat er dus een verband is tussen de variabelen.

In Python gebruiken we daarvoor volgende Scipy-functies:

- `stats.chi2.sf(x)` - de rechterstaartkans van `x` (of "survival function")
- `stats.chi2.isf(q)` - de inverse functie van `sf(x)`, m.a.w. een getal `x` berekenen waarvoor geldt dat de rechterstaartkans precies `q` is.

Een plot voor het geval dat $\alpha = 0.05$ (een vaak gekozen waarde voor het significantieniveau) en 4 vrijheidsgraden:

In [ ]:
# Rechterstaartkans in de chi-kwadraatverdeling

# x-waarden:
x = np.linspace(0, 15, num=100)
# kansdichtheid van de chi-kwadraatverdeling met 4 vrijheidsgraden:
y = stats.chi2.pdf(x, df=4)
# het getal q waarvoor geldt dat de rechterstaartkans precies 5% is:
q = stats.chi2.isf(.05, df=4)

fig, tplot = plt.subplots(1, 1)
tplot.plot(x, y)                      # kansdichtheid
tplot.fill_between(x, y, where=x>=q,  # kritieke gebied
    color='lightblue')
tplot.axvline(q)                      # kritieke grenswaarde

### Toetsingsprocedure

Formeel verloopt de procedure van de $\chi^2$ onafhankelijkheidstoets als volgt:

1. Formuleer de hypotheses:
   - $H_0$: Er is geen verband tussen de variabelen (de verschillen tussen geobserveerde en verwachte waarden zijn klein)
   - $H_1$: Er is een verband tussen de variabelen (de verschillen zijn groot)
2. Bepaal het significantieniveau $\alpha$
3. Bereken de waarde van de teststatistiek (of toetsingsgrootheid) in de steekproef (hier: $\chi^2$).
4. Gebruik een van de volgende methoden (op basis van het aantal vrijheidsgraden $df = (r-1) \times (k-1)$):
   1. Bepaal de kritieke grenswaarde $g$ zodat $P(\chi^2 > g) = \alpha$
   2. Bereken de overschrijdingskans $p$
5. Trek een besluit op basis van de uitkomst:
   1. $\chi^2 < g$: $H_0$ niet verwerpen; $\chi^2 > g$: $H_0$ verwerpen
   2. $p > \alpha$: $H_0$ niet verwerpen; $p < \alpha$: $H_0$ verwerpen

In Python kunnen we dit als volgt berekenen:

In [ ]:
alpha = .05
dimensions = observed.shape
dof = (dimensions[0]-1) * (dimensions[1]-1)

print("Chi-kwadraat         : %.4f" % chi_squared)
print("Vrijheidsgraden      : %d" % dof)

# Berekenen kritieke grenswaarde
g = stats.chi2.isf(alpha, df = dof)
print("Kritieke grenswaarde : %.4f" % g)

# Berekenen overschrijdingskans
p = stats.chi2.sf(chi_squared, df=dof)
print("Overschrijdingskans  : %.4f" % p)

Eigenlijk hoeven we deze rekenregels om chi-kwadraat, de kritieke grenswaarde en de overschrijdingskans te bepalen niet te onthouden. In SciPy zit al een functie waarmee we op basis van een kruistabel rechtstreeks de chi-kwadraat en de p-waarde kunnen berekenen:

In [ ]:
# Chi-kwadraat onanfhankelijkheidstoets op basis van een kruistabel
observed = pd.crosstab(rlanders.Survey, rlanders.Gender)
chi2, p, df, expected = stats.chi2_contingency(observed)

print("Chi-kwadraat   : %.4f" % chi2)
print("Vrijheidsgraden: %d" % df)
print("P-waarde       : %.4f" % p)

Zoals je ziet is de uitkomst voor de $p$-waarde dezelfde als in onze uitgewerkte berekeningen!

Een grafische voorstelling van onze casus:

In [ ]:
# Is er een verband tussen Gender en Survey?

# x-waarden:
x = np.linspace(0, 15, num=100)
# kansdichtheid van de chi-kwadraatverdeling met 4 vrijheidsgraden:
y = stats.chi2.pdf(x, df=dof)
# het getal q waarvoor geldt dat de rechterstaartkans precies 5% is:

fig, tplot = plt.subplots(1, 1)
tplot.plot(x, y)                     # kansdichtheid
tplot.fill_between(x, y, where=x>=q, # kritieke gebied
    color='lightblue')
tplot.axvline(q)                     # kritieke grenswaarde
tplot.axvline(chi2, color='orange')  # chi-kwadraat

We zien dat $\chi^2$ ruim binnen het aanvaardingsgebied ligt. De $p$-waarde is ook groter dan $\alpha$. We kunnen dus de nulhypothese niet verwerpen en besluiten dat er op basis van deze steekproef geen reden is om aan te nemen dat er een significant verschil is tussen de antwoorden van vrouwen en mannen op de enquêtevraag.

## Aanpassingstoets (goodness-of-fit test)

Stel dat we in een steekproef van superhelden bijhouden van welk type ze zijn en dat we ook weten hoe vaak elk type in de gehele populatie voorkomt (als percentage). Hieronder zijn de absolute frequenties in de steekproef $o_i$ en de verwachte relatieve frequenties $\pi_i$ in de populatie gegeven:

In [ ]:
types =               ['mutant', 'human', 'alien', 'god', 'demon']
observed =   np.array([   127,      75,      98,     27,     73])
expected_p = np.array([   .35,     .17,     .23,    .08,    .17])

De vraag is nu: is deze steekproef representatief voor de populatie? Komt elk type in de steekproef voor in verhouding tot het verwachge percentage in de populatie als geheel?

### Testprocedure

Om dit soort vragen te beantwoorden is een aanpassingstoets (goodness-of-fit test) geschikt. De procedure verloopt als volgt:

1. Formuleer de hypotheses:
   - $H_0$: De steekproef is representatief voor de populatie, d.w.z. de frequentie van elke klasse binnen de steekproef komt goed overeen met die in de populatie.
   - $H_1$: De steekproef is *niet* representatief voor de populatie, d.w.z. de verschillen met de verwachte frequenties is te groot.
2. Bepaal het significantieniveau $\alpha$
3. Bereken de waarde van de teststatistiek (of toetsingsgrootheid) in de steekproef (hier: $\chi^2$).
4. Gebruik een van de volgende methoden (op basis van het aantal vrijheidsgraden $df = (k-1)$ met $k$ het aantal klassen in de steekproef):
   1. Bepaal de kritieke grenswaarde $g$ zodat $P(\chi^2 > g) = \alpha$
   2. Bereken de overschrijdingskans $p$
5. Trek een besluit op basis van de uitkomst:
   1. $\chi^2 < g$: $H_0$ niet verwerpen; $\chi^2 > g$: $H_0$ verwerpen
   2. $p > \alpha$: $H_0$ niet verwerpen; $p < \alpha$: $H_0$ verwerpen

Een plot van deze casus, met berekining van $p$ en $g$:

In [ ]:
alpha = 0.05               # Significantieniveau
n = sum(observed)          # Steekproefgrootte
k = len(observed)          # Aantal categorieën
dof = k - 1                # Aantal vrijheidsgraden
expected = expected_p * n  # Verwachte absolute frequenties in de steekproef
g = stats.chi2.isf(alpha, df=dof)  # Kritieke grenswaarde

# Goodness-of-fit-test in Python:
chi2, p = stats.chisquare(f_obs=observed, f_exp=expected)

print("Significantieniveau  ⍺ = %.2f" % alpha)
print("Steekproefgrootte    n = %d" % n)
print("k = %d; df = %d" % (k, dof))
print("Chi-kwadraat        χ² = %.4f" % chi2)
print("Kritieke grenswaarde g = %.4f" % g)
print("Overschrijdingskans  p = %.4f" % p)

In [ ]:
# Plot van de casus:
# x-waarden:
x = np.linspace(0, 15, num=100)
# kansdichtheid van de chi-kwadraatverdeling met 4 vrijheidsgraden:
y = stats.chi2.pdf(x, df=dof)
# het getal q waarvoor geldt dat de rechterstaartkans precies 5% is:

fig, tplot = plt.subplots(1, 1)
tplot.plot(x, y)                     # kansdichtheid
tplot.fill_between(x, y, where=x>=q, # kritieke gebied
    color='lightblue')
tplot.axvline(q)                     # kritieke grenswaarde
tplot.axvline(chi2, color='orange')  # chi-kwadraat

We zien dat de $\chi^2$ in de steekproef links van de kritieke grenswaarde ligt, dus in het aanvaardingsgebied. We kunnen de nulhypothese dus niet verwerpen en besluiten dat de steekproef representatief is voor de populatie.

## Gestandaardiseerde residuën

Na het uitvoeren van een chi-kwadraattoets is het vaak ook interessant om te weten in welke categorieën de grootste afwijkingen voorkomen. Je zou voor elke cel in de kruistabel (of vector) kunnen kijken naar de waarde $\frac{(o-e)^2}{e}$ (wat in de berekening van $\chi^2$ gebruikt wordt). Daaruit kan je echter niet opmaken in hoeverre de afwijkingen echt als "extreem" beschouwd kunnen worden. Hiervoor gebruiken we zgn. *gestandaardiseerde residuën*.

We introduceren dit begrip a.h.v. een voorbeeld:

Stel dat in een bepaald onderzoek gekeken wordt naar gezinnen met 5 kinderen. Er is een steekproef genomen van 1022 gezinnen, en deze worden in categorieën verdeeld naargelang het aantal jongens in het gezin. De frequenties worden hieronder gegeven in een Pandas DataFrame die we zelf aanmaken en stelselmatig zullen uitbreiden.

In [ ]:
# Dataframe met 2 kolommen:
#  - het aantal jongens in het gezin (index)
#  - aantal families in de steekproef met dat aantal jongens
families = pd.DataFrame(
    np.array(
        [[0,  58],
         [1, 149],
         [2, 305],
         [3, 303],
         [4, 162],
         [5,  45]]),
    columns=[ 'num_boys', "observed"])
families.set_index(['num_boys'])
n = families.observed.sum() # Steekproefgrootte

Wat is nu het verwachte aantal gezinnen met het gegeven aantal jongens? Laten we veronderstellen dat er evenveel kans is om een jongen of meisje te verwekken. Dan kan je de verwachte relatieve frequenties als volgt berekenen:

$\pi_i = (0.5)^i (1-0.5)^{5-i} \binom{5}{i}$

Waarom dat zo is, is niet relevant voor het onderwerp waar we nu mee bezig zijn...

De verwachte absolute frequenties in de steekproef kan je dan berekenen als $e_i = n \pi_i$.

We passen dit toe op onze casus:

In [ ]:
from scipy.special import binom # binomiaal-functie

# de kans op een jongen
prob_boy = .5
# Nieuwe kolom aan het DataFrame toevoegen voor de verwachte percentages,
# op basis van de formule
families['expected_p'] = binom(5, families.num_boys) * prob_boy**families.num_boys * prob_boy**(5-families.num_boys)
# Verwachte absolute frequenties in de steekproef
families['expected'] = expected_p * n
families

We voeren nu de chi-kwadraat aanpassingstoets uit. Hier is als significantieniveau $\alpha = 0.01$ gekozen.

In [ ]:
alpha=0.01                         # significantieniveau
dof=len(families)-1                # aantal vrijheidsgraden
g = stats.chi2.isf(alpha, df=dof)  # Kritieke grenswaarde
# Chi-kwadraattoets uitvoeren, χ² en p berekenen
chi2, p = stats.chisquare(f_obs=families.observed, f_exp=families.expected)

print("Chi-kwadraat        χ² = %.4f" % chi2)
print("Kritieke grenswaarde g = %.4f" % g)
print("Overschrijdingskans  p = %f"   % p)

Dit is een erg kleine overschrijdingskans. De chi-kwadraat ligt ook rechts van de kritieke grenswaarde. Dat betekent dat we de nulhypothese kunnen verwerpen. Grafisch:

In [ ]:
# Plot van de casus:
# x-waarden:
x = np.linspace(0, 30, num=200)
# kansdichtheid van de chi-kwadraatverdeling met 4 vrijheidsgraden:
y = stats.chi2.pdf(x, df=dof)
# het getal q waarvoor geldt dat de rechterstaartkans precies 5% is:

fig, tplot = plt.subplots(1, 1)
tplot.plot(x, y)                     # kansdichtheid
tplot.fill_between(x, y, where=x>=q, # kritieke gebied
    color='lightblue')
tplot.axvline(q)                     # kritieke grenswaarde
tplot.axvline(chi2, color='orange')  # chi-kwadraat

Ons besluit is dus dat deze steekproef niet representatief is voor de populatie!

De formule voor het berekenen van de gestandaardiseerde residuën is:

$r_i = \frac{o_i-e_i}{\sqrt{e_i (1-\pi_i)}}$

We voegen een kolom toe aan onze tabel (probeer de code te vergelijken met de formule!):

In [ ]:
families['stdres'] = (families.observed - families.expected) / np.sqrt(families.expected * (1 - families.expected_p))
families

Gestandaardiseerde residuën zijn een maat om aan te geven in hoeverre een bepaalde categorie over- of ondervertegenwoordigd is in de steekproef. Een waarde van 0 krijg je als de geobserveerde frequentie gelijk is aan de verwachte. Een negatieve waarde krijg je als er minder observaties zijn dan verwacht en een positieve als er meer zijn. Zolang $r_i \in [-2, 2]$, beschouwen we de verschillen als toevallige steekproeffouten. Een waarde $r_i < -2$ duidt op ondervertegenwoordiging van deze categorie, $r_i > 2$ op oververtegenwoordiging.

In de steekproef zijn dus de gezinnen met enkel meisjes en enkel jongens oververtegenwoordigd. In de praktijk zouden onderzoekers er voor kunnen kiezen om aselect een aantal observaties in deze categorieën te schrappen uit het onderzoek, zodat de steekproef representatief wordt voor de populatie.

## De regel van Cochran

Een chi-kwadraattoets kan enkel goede resultaten opleveren als je voldoende observaties hebt in elke categorie. De statisticus Cochran (1954) formuleerde een vuistregel om te bepalen wat exact *genoeg* is op kruistabellen die groter zijn dan 2x2:

- Alle verwachte waarden moeten minstens 1 zijn
- Hoogstens 20% van de verwachte waarden mag kleiner dan 5 zijn



## TODO

- functie voor toepassen regel Cochran?
- toepassen regel Cochran op de casussen?
- residuën in een kruistabel, ahv statsmodels
   - <https://www.statsmodels.org/stable/contingency_tables.html?highlight=residuals>
   - <https://stackoverflow.com/questions/20453729/what-is-the-equivalent-of-r-data-chisqresiduals-in-python>
